In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Chemin vers le dossier contenant les images
data_dir = 'chemin/vers/votre/dossier'

# Liste des images et des étiquettes
images = []
labels = []

# Parcourir les fichiers dans le dossier
for filename in os.listdir(data_dir):
    if filename.endswith('.png'):
        # Extraire les étiquettes du nom de fichier
        color, piece, _ = filename.split('_')
        label = f"{color}_{piece}"

        # Charger l'image
        img_path = os.path.join(data_dir, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(64, 64))
        img = tf.keras.preprocessing.image.img_to_array(img)

        # Ajouter l'image et l'étiquette aux listes
        images.append(img)
        labels.append(label)

# Convertir les listes en tableaux numpy
images = np.array(images)
labels = np.array(labels)

# Encoder les étiquettes
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = tf.keras.utils.to_categorical(labels_encoded)

# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

# Créer un générateur d'images pour l'augmentation des données
datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)

# Construire le modèle CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_val, y_val))

# Évaluer le modèle
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


In [ ]:
# Fonction pour afficher les images avec les prédictions et les vraies réponses
def display_images_with_predictions_and_truth(model, images, true_labels, labels_encoder, num_images=5):
    # Sélectionner un sous-ensemble aléatoire d'images
    indices = np.random.choice(range(len(images)), num_images, replace=False)
    selected_images = images[indices]
    selected_true_labels = true_labels[indices]

    # Prédire les étiquettes pour les images sélectionnées
    predictions = model.predict(selected_images)
    predicted_labels = labels_encoder.inverse_transform(np.argmax(predictions, axis=1))
    true_labels_decoded = labels_encoder.inverse_transform(np.argmax(selected_true_labels, axis=1))

    # Afficher les images avec leurs prédictions et les vraies réponses
    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(selected_images[i] / 255.0)  # Normaliser les valeurs des pixels pour l'affichage
        plt.title(f"Pred: {predicted_labels[i]}\nTrue: {true_labels_decoded[i]}")
        plt.axis('off')
    plt.show()

# Exemple d'utilisation
# Assurez-vous que 'model' est votre modèle entraîné, 'X_val' est votre ensemble de validation,
# 'y_val' est les vraies étiquettes pour l'ensemble de validation,
# et 'label_encoder' est l'encodeur d'étiquettes utilisé pour encoder les étiquettes.
display_images_with_predictions_and_truth(model, X_val, y_val, label_encoder)


In [3]:
# Load your trained Keras model
model = tf.keras.models.load_model("chess_piece_model.keras")

# Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
mobile_model_path = os.path.join(
    '..',
    'mobile_application',
    'assets',
    'chess_piece_model.tflite'
)

with open(mobile_model_path, "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\alexa\AppData\Local\Temp\tmpy8p9u8hc\assets


INFO:tensorflow:Assets written to: C:\Users\alexa\AppData\Local\Temp\tmpy8p9u8hc\assets


Saved artifact at 'C:\Users\alexa\AppData\Local\Temp\tmpy8p9u8hc'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 14), dtype=tf.float32, name=None)
Captures:
  1896638039440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1896638040976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1896638041552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1896638042128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1896638041360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1896638042896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1896638038864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1896638043856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1896638043280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1896638045392: TensorSpec(shape=(), dtype=tf.resource, name=None)
